In [2]:
classes = [0, 1]
num_classes = len(classes)

In [3]:
from dataset import ExpressionDataset, get_transform
from torch.utils.data import DataLoader

train_dataset = ExpressionDataset(
    "../../dataset/train/images","../../dataset/train/labels","train", get_transform(True,800)
)
val_dataset = ExpressionDataset(
    "../../dataset/valid/images","../../dataset/valid/labels", "valid", get_transform(False,800)
)
test_dataset = ExpressionDataset(
    "../../dataset/test/images", "../../dataset/test/labels", "test",get_transform(False,800)
)


train_loader = DataLoader(
    train_dataset,
    # num_workers=4,
    pin_memory=True,
    batch_size=1,
    shuffle=True,
    collate_fn=lambda x: tuple(zip(*x)),
)
val_loader = DataLoader(
    val_dataset,
    # num_workers=4,
    pin_memory=True,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: tuple(zip(*x)),
)
test_loader = DataLoader(
    test_dataset,
    # num_workers=4,
    pin_memory=True,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: tuple(zip(*x)),
)

In [4]:
from model import FasterRCNN
from train import training, testing
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

model = FasterRCNN(num_classes)
model.to(device)
training(model, train_loader, val_loader, num_epochs=30, device='cpu')

cuda


c:\Users\fardi\.conda\envs\ml\lib\site-packages\torch\cuda\__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 1060 which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
c:\Users\fardi\.conda\envs\ml\lib\site-packages\torch\cuda\__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
c:\Users\fardi\.conda\envs\ml\lib\site-packages\torch\cuda\__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 1060 with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 1060 GPU with PyTorch, please check the instructions at https://pytorch.org/ge

KeyboardInterrupt: 

In [ ]:
model = FasterRCNN(num_classes)
model.load_state_dict(torch.load("../../results/part1/best_model11.pth"))
model.to(device)
test_detections, test_targets = testing(model, test_loader, device)

In [ ]:
from draw_boxes import draw_boxes
draw_boxes("../../dataset/test/images/618.png", test_detections[118]['boxes'])

In [ ]:
import csv
import os
from torchvision.transforms import ToTensor
from PIL import Image

output_csv = "../../results/part1/output.csv"
with open(output_csv, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["image_id", "x", "y", "width", "height"])

    for i in range(0,200):
      boxes = test_detections[i]['boxes'].cpu()
      for box in boxes:
          x1, y1, x2, y2 = box
          width = x2 - x1
          height = y2 - y1
          writer.writerow([i +500, int(x1), int(y1), int(width), int(height)])
